In [2]:
import os

import allel
import numpy as np
import pandas as pd
import ast
import math
import re
import itertools
import random
import json
import matplotlib
from matplotlib.pyplot import plot as plt
from pylab import *
from scipy.spatial.distance import squareform

mpl.use("pgf")
# activate latex text rendering
rc('text', usetex=True)
rc('axes', linewidth=2)
rc('font', weight='bold')
mpl.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
    'text.latex.preamble':r'\usepackage{sfmath} \boldmath'
})
%matplotlib inline
import seaborn as sns
sns.set_theme(style="whitegrid")
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.model_selection import train_test_split
from tqdm import tqdm
# sns.set_theme(style="ticks")


In [43]:
# load data
root_dir = './data/'
# load data
new_data_header = ""
# get header
with open(root_dir + "HLA.recode.vcf", 'r') as f_in:
    # skip info
    for line_num in range(252):
        f_in.readline()

    new_data_header = f_in.readline()
# load data

# load genotype
genotypes = pd.read_csv(root_dir + "HLA.recode.vcf",
                        comment='#', sep='\t',
                        names=new_data_header.strip().split('\t'),
                        header=None).iloc[:, 9:].T

headers = genotypes.columns[:]
genotypes


,0,1,2,3,4,5,6,7,8,9,...,7151,7152,7153,7154,7155,7156,7157,7158,7159,7160
HG00096,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
HG00097,0|0,1|0,1|0,0|0,1|0,0|0,0|0,0|0,0|0,1|0,...,0|1,0|1,0|1,0|1,0|0,0|1,0|1,0|1,0|1,0|0
HG00099,0|0,0|1,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,...,1|1,1|1,1|1,1|1,0|0,1|1,1|1,1|1,1|1,0|0
HG00100,0|0,0|1,0|1,0|0,0|1,0|0,0|0,0|0,0|0,0|1,...,0|1,0|1,0|1,0|1,0|0,0|1,0|1,0|1,0|1,0|0
HG00101,0|0,1|1,1|0,0|0,1|0,0|0,0|0,0|0,0|0,1|0,...,1|0,1|0,1|0,1|0,0|0,1|0,1|0,1|0,1|0,0|0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NA21137,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
NA21141,0|0,1|1,1|1,0|0,1|1,0|0,0|0,0|0,0|0,1|1,...,1|0,1|0,0|0,1|0,1|0,0|0,0|0,1|0,0|0,0|0
NA21142,0|0,1|1,0|1,0|0,0|1,0|0,0|0,0|0,0|0,0|1,...,0|1,0|1,0|1,0|1,0|0,0|1,0|1,0|1,0|1,0|0
NA21143,0|0,1|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,...,1|1,1|1,1|1,1|1,0|0,1|1,1|1,1|1,1|1,0|0


In [44]:
ped_file = 'integrated_call_samples.20130502.ALL.ped'
pedigree = pd.read_csv(root_dir+ped_file, sep='\t', index_col='Individual ID')
pedigree.head()

,Family ID,Paternal ID,Maternal ID,Gender,Phenotype,Population,Relationship,Siblings,Second Order,Third Order,Children,Other Comments
Individual ID,,,,,,,,,,,,
HG00096,HG00096,0,0,1,0,GBR,unrel,0,0,0,0,0
HG00097,HG00097,0,0,2,0,GBR,unrel,0,0,0,0,0
HG00098,HG00098,0,0,1,0,GBR,unrel,0,0,0,0,0
HG00099,HG00099,0,0,2,0,GBR,unrel,0,0,0,0,0
HG00100,HG00100,0,0,2,0,GBR,unrel,0,0,0,0,0


In [70]:
Y_train = pedigree.loc[genotypes.index]['Population']
X = genotypes[genotypes.index.isin(Y_train.index)]
X = X.replace({
    '0|0': 0,
    '0|1': 1,
    '1|0': 1,
    '1|1': 2
})
X.shape

(2504, 7161)

In [71]:

def freq_mapper(g):
  return 0 if g == 0 else 1 if g==1 or g==2 else 2

def bin_label_to_maf(g):
    return g/10

def maf_mapper(g):
  return min(g, 1-g)

_X = X.applymap(freq_mapper)
_X = _X.sum(axis=0)/(2*X.shape[0])
mafs = _X.map(maf_mapper)

bin_labels = np.digitize(mafs, bins=[0, 0.1, 0.2, 0.3, 0.4, 0.5], right=True)
maf_bin_labels = np.vectorize(bin_label_to_maf)(bin_labels)
bin_general_labels, bin_counts = np.unique(bin_labels, return_counts=True)
bin_general_labels, bin_counts


(array([1, 2, 3, 4, 5], dtype=int64),
 array([1904, 1710, 1222, 1227, 1098], dtype=int64))

In [72]:
def bin_label_mapper(bin_label):
    if bin_label == 1:
        return "$0 \le MAF < 0.1 $"
    elif bin_label == 2:
        return "$0.1 \le MAF < 0.2 $"
    elif bin_label == 3:
        return "$0.2 \le MAF < 0.3 $"
    elif bin_label == 4:
        return "$0.3 \le MAF < 0.4 $"
    else:
        return "$0.4 \le MAF \le 0.5 $"

bin_labels_str = list(map(bin_label_mapper, bin_labels))
bin_labels_str[:2]

['$0 \\le MAF < 0.1 $', '$0.3 \\le MAF < 0.4 $']

In [73]:
def iqs_score(data_imp, data_obs):
    #predict_onehot = SCDA.predict(test_X_missing[:, :, :])

    print('data_imp:', data_imp.shape)

    # data_imp: rows (snps); cols (samples)
    data_imp_012 = data_imp.transpose(1, 0)

    # data_imp_012 = data_imp - 1


    print('data_imp_012:', data_imp_012.shape)

    snp_cnt = data_imp_012.shape[0]
    sample_cnt = data_imp_012.shape[1]
    print('snp_cnt:', snp_cnt)
    print('sample_cnt:', sample_cnt)

    data_obs_012 = data_obs.transpose(1, 0)
    print('data_obs_012:', data_obs_012.shape)

    # data_obs_012 = data_obs - 1


    iqs = [0 for _ in range(snp_cnt)]


    for i in tqdm(range(snp_cnt)):
        iqs_cnt = [[0 for x in range(3)] for y in range(3)]

        for j in range(sample_cnt):
            if(data_obs_012[i][j]==0) and (data_imp_012[i][j]==0):
                iqs_cnt[0][0] += 1
            elif(data_obs_012[i][j]==1) and (data_imp_012[i][j]==0):
                iqs_cnt[0][1] += 1
            elif(data_obs_012[i][j]==2) and (data_imp_012[i][j]==0):
                iqs_cnt[0][2] += 1

            elif(data_obs_012[i][j]==0) and (data_imp_012[i][j]==1):
                iqs_cnt[1][0] += 1
            elif(data_obs_012[i][j]==1) and (data_imp_012[i][j]==1):
                iqs_cnt[1][1] += 1
            elif(data_obs_012[i][j]==2) and (data_imp_012[i][j]==1):
                iqs_cnt[1][2] += 1

            elif(data_obs_012[i][j]==0) and (data_imp_012[i][j]==2):
                iqs_cnt[2][0] += 1
            elif(data_obs_012[i][j]==1) and (data_imp_012[i][j]==2):
                iqs_cnt[2][1] += 1
            elif(data_obs_012[i][j]==2) and (data_imp_012[i][j]==2):
                iqs_cnt[2][2] += 1

        po = (iqs_cnt[0][0] + iqs_cnt[1][1] + iqs_cnt[2][2])/sample_cnt

        pc_rows_0 = iqs_cnt[0][0] + iqs_cnt[0][1] + iqs_cnt[0][2]
        pc_rows_1 = iqs_cnt[1][0] + iqs_cnt[1][1] + iqs_cnt[1][2]
        pc_rows_2 = iqs_cnt[2][0] + iqs_cnt[2][1] + iqs_cnt[2][2]

        pc_cols_0 = iqs_cnt[0][0] + iqs_cnt[1][0] + iqs_cnt[2][0]
        pc_cols_1 = iqs_cnt[0][1] + iqs_cnt[1][1] + iqs_cnt[2][1]
        pc_cols_2 = iqs_cnt[0][2] + iqs_cnt[1][2] + iqs_cnt[2][2]

        pc = (pc_rows_0*pc_cols_0 + pc_rows_1*pc_cols_1 + pc_rows_2*pc_cols_2)/(sample_cnt**2)


        if pc == 1:
            iqs[i] = 1
        else:
            iqs[i] = (po - pc)/(1 - pc)


    print('iqs mean:', np.mean(iqs))
    return iqs, np.mean(iqs)

In [74]:
columns = ["Fold", "Missing Rate", "IQS", "Method", "MAF"]
dfs = []
N_SPLITS=3

In [75]:
kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=2022)
fold = 0
_x = X[X.index.isin(Y_train.index)].to_numpy()
for train_index, test_index in kf.split(_x):
  fold += 1
  print(f"Fold: {fold}")
  y = _x[test_index]
  for missing_perc in [0.01,
                       0.05,
                       0.1,
                       0.2]:
    print(f"Missing rate: {missing_perc}")
    file_name = f"./experimental_results/hla/maf/Minimac4/preds_mr_{missing_perc}_rs_{fold}_.csv"
    result_genotypes = pd.read_csv(file_name, index_col=0)
    result_genotypes = result_genotypes.replace({
        '0|0': 0,
        '0|1': 1,
        '1|0': 1,
        '1|1': 2
    })
    yhat = result_genotypes.to_numpy()
    assert y.shape == yhat.shape
    iqs_a, iqs_mean_a = iqs_score(yhat, y)

    # print('iqs_mean:', iqs_mean_a)
    # print('iqs:', iqs_a)

    dataframe = pd.DataFrame(np.stack([np.full((len(iqs_a)), fold),
                                       np.full((len(iqs_a)), missing_perc),
                                       np.array(iqs_a),
                                       np.full((len(iqs_a)), "Minimac4"),
                                       np.array(bin_labels_str)], axis=1), columns=columns)
    dfs.append(dataframe)

Fold: 1
Missing rate: 0.01
data_imp: (835, 7161)
data_imp_012: (7161, 835)
snp_cnt: 7161
sample_cnt: 835
data_obs_012: (7161, 835)


100%|██████████| 7161/7161 [00:15<00:00, 448.20it/s]


iqs mean: 0.9965428517051288
Missing rate: 0.05
data_imp: (835, 7161)
data_imp_012: (7161, 835)
snp_cnt: 7161
sample_cnt: 835
data_obs_012: (7161, 835)


100%|██████████| 7161/7161 [00:15<00:00, 451.45it/s]


iqs mean: 0.9960551361826521
Missing rate: 0.1
data_imp: (835, 7161)
data_imp_012: (7161, 835)
snp_cnt: 7161
sample_cnt: 835
data_obs_012: (7161, 835)


...

iqs mean: 0.996074819077267
Missing rate: 0.1
data_imp: (834, 7161)
data_imp_012: (7161, 834)
snp_cnt: 7161
sample_cnt: 834
data_obs_012: (7161, 834)


100%|██████████| 7161/7161 [00:15<00:00, 454.83it/s]


iqs mean: 0.9954696281558381
Missing rate: 0.2
data_imp: (834, 7161)
data_imp_012: (7161, 834)
snp_cnt: 7161
sample_cnt: 834
data_obs_012: (7161, 834)


100%|██████████| 7161/7161 [00:15<00:00, 454.96it/s]

iqs mean: 0.9942150404104487


In [76]:
kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=2022)
fold = 0
_x = X[X.index.isin(Y_train.index)].to_numpy()[:, :-1]
for train_index, test_index in kf.split(_x):
  fold += 1
  print(f"Fold: {fold}")
  y = _x[test_index]
  for missing_perc in [0.01,
                       0.05,
                       0.1,
                       0.2]:
    print(f"Missing rate: {missing_perc}")
    file_name = f"./experimental_results/hla/maf/HLA_DEEP/HLA_DEEP_preds_mixed_mr_{missing_perc}_rs_{fold}_.csv"
    result_genotypes = pd.read_csv(file_name, index_col=0)
    result_genotypes = result_genotypes.replace({
        '0|0': 0,
        '0|1': 1,
        '1|0': 1,
        '1|1': 2
    })
    yhat = result_genotypes.to_numpy()
    assert y.shape == yhat.shape
    iqs_a, iqs_mean_a = iqs_score(yhat, y)

    # print('iqs_mean:', iqs_mean_a)
    # print('iqs:', iqs_a)

    dataframe = pd.DataFrame(np.stack([np.full((len(iqs_a)), fold),
                                       np.full((len(iqs_a)), missing_perc),
                                       np.array(iqs_a),
                                       np.full((len(iqs_a)), "HLA*DEEP"),
                                       np.array(bin_labels_str[:-1])], axis=1), columns=columns)
    dfs.append(dataframe)

Fold: 1
Missing rate: 0.01
data_imp: (835, 7160)
data_imp_012: (7160, 835)
snp_cnt: 7160
sample_cnt: 835
data_obs_012: (7160, 835)


100%|██████████| 7160/7160 [00:15<00:00, 456.54it/s]


iqs mean: 0.9884320858485816
Missing rate: 0.05
data_imp: (835, 7160)
data_imp_012: (7160, 835)
snp_cnt: 7160
sample_cnt: 835
data_obs_012: (7160, 835)


100%|██████████| 7160/7160 [00:15<00:00, 448.49it/s]


iqs mean: 0.987841894462968
Missing rate: 0.1
data_imp: (835, 7160)
data_imp_012: (7160, 835)
snp_cnt: 7160
sample_cnt: 835
data_obs_012: (7160, 835)


...

iqs mean: 0.9869934365738271
Missing rate: 0.1
data_imp: (834, 7160)
data_imp_012: (7160, 834)
snp_cnt: 7160
sample_cnt: 834
data_obs_012: (7160, 834)


100%|██████████| 7160/7160 [00:15<00:00, 458.24it/s]


iqs mean: 0.9862765417714316
Missing rate: 0.2
data_imp: (834, 7160)
data_imp_012: (7160, 834)
snp_cnt: 7160
sample_cnt: 834
data_obs_012: (7160, 834)


100%|██████████| 7160/7160 [00:15<00:00, 456.06it/s]

iqs mean: 0.9845855992517462


In [77]:
kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=2022)
fold = 0
_x = X[X.index.isin(Y_train.index)].to_numpy()[:, :-1]
for train_index, test_index in kf.split(_x):
  fold += 1
  print(f"Fold: {fold}")
  y = _x[test_index]
  for missing_perc in [0.01,
                       0.05,
                       0.1,
                       0.2]:
    print(f"Missing rate: {missing_perc}")
    file_name = f"./experimental_results/hla/maf/SCDA/SCDA_preds_mixed_mr_{missing_perc}_rs_{fold}_.csv"
    result_genotypes = pd.read_csv(file_name, index_col=0)
    result_genotypes = result_genotypes.replace({
        '0|0': 0,
        '0|1': 1,
        '1|0': 1,
        '1|1': 2
    })
    yhat = result_genotypes.to_numpy()
    assert y.shape == yhat.shape
    iqs_a, iqs_mean_a = iqs_score(yhat, y)

    # print('iqs_mean:', iqs_mean_a)
    # print('iqs:', iqs_a)

    dataframe = pd.DataFrame(np.stack([np.full((len(iqs_a)), fold),
                                       np.full((len(iqs_a)), missing_perc),
                                       np.array(iqs_a),
                                       np.full((len(iqs_a)), "SCDA+"),
                                       np.array(bin_labels_str[:-1])], axis=1), columns=columns)
    dfs.append(dataframe)

Fold: 1
Missing rate: 0.01
data_imp: (835, 7160)
data_imp_012: (7160, 835)
snp_cnt: 7160
sample_cnt: 835
data_obs_012: (7160, 835)


100%|██████████| 7160/7160 [00:15<00:00, 452.14it/s]


iqs mean: 0.9966600279620402
Missing rate: 0.05
data_imp: (835, 7160)
data_imp_012: (7160, 835)
snp_cnt: 7160
sample_cnt: 835
data_obs_012: (7160, 835)


100%|██████████| 7160/7160 [00:15<00:00, 451.36it/s]


iqs mean: 0.9955595741845441
Missing rate: 0.1
data_imp: (835, 7160)
data_imp_012: (7160, 835)
snp_cnt: 7160
sample_cnt: 835
data_obs_012: (7160, 835)


...

iqs mean: 0.9952392153006936
Missing rate: 0.1
data_imp: (834, 7160)
data_imp_012: (7160, 834)
snp_cnt: 7160
sample_cnt: 834
data_obs_012: (7160, 834)


100%|██████████| 7160/7160 [00:15<00:00, 460.89it/s]


iqs mean: 0.9938139116443612
Missing rate: 0.2
data_imp: (834, 7160)
data_imp_012: (7160, 834)
snp_cnt: 7160
sample_cnt: 834
data_obs_012: (7160, 834)


100%|██████████| 7160/7160 [00:15<00:00, 457.05it/s]

iqs mean: 0.9906947624174149


In [78]:
kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=2022)
fold = 0
_x = X[X.index.isin(Y_train.index)].to_numpy()[:, :-1]
for train_index, test_index in kf.split(_x):
  fold += 1
  print(f"Fold: {fold}")
  y = _x[test_index]
  for missing_perc in [0.01,
                       0.05,
                       0.1,
                       0.2]:
    print(f"Missing rate: {missing_perc}")
    file_name = f"./experimental_results/hla/maf/AE/AE_preds_mixed_mr_{missing_perc}_fold_{fold}_.csv"
    result_genotypes = pd.read_csv(file_name, index_col=0)
    result_genotypes = result_genotypes.replace({
        0 : 0,
        1 : 1,
        2 : 1,
        3 : 2
    })
    yhat = result_genotypes.to_numpy()
    assert y.shape == yhat.shape
    iqs_a, iqs_mean_a = iqs_score(yhat, y)

    # print('iqs_mean:', iqs_mean_a)
    # print('iqs:', iqs_a)

    dataframe = pd.DataFrame(np.stack([np.full((len(iqs_a)), fold),
                                       np.full((len(iqs_a)), missing_perc),
                                       np.array(iqs_a),
                                       np.full((len(iqs_a)), "AE"),
                                       np.array(bin_labels_str[:-1])], axis=1), columns=columns)
    dfs.append(dataframe)

Fold: 1
Missing rate: 0.01
data_imp: (835, 7160)
data_imp_012: (7160, 835)
snp_cnt: 7160
sample_cnt: 835
data_obs_012: (7160, 835)


100%|██████████| 7160/7160 [00:15<00:00, 455.70it/s]


iqs mean: 0.9957168617064006
Missing rate: 0.05
data_imp: (835, 7160)
data_imp_012: (7160, 835)
snp_cnt: 7160
sample_cnt: 835
data_obs_012: (7160, 835)


100%|██████████| 7160/7160 [00:15<00:00, 450.16it/s]


iqs mean: 0.9882750538183256
Missing rate: 0.1
data_imp: (835, 7160)
data_imp_012: (7160, 835)
snp_cnt: 7160
sample_cnt: 835
data_obs_012: (7160, 835)


...

iqs mean: 0.9880347575102788
Missing rate: 0.1
data_imp: (834, 7160)
data_imp_012: (7160, 834)
snp_cnt: 7160
sample_cnt: 834
data_obs_012: (7160, 834)


100%|██████████| 7160/7160 [00:15<00:00, 450.93it/s]


iqs mean: 0.9809134473116534
Missing rate: 0.2
data_imp: (834, 7160)
data_imp_012: (7160, 834)
snp_cnt: 7160
sample_cnt: 834
data_obs_012: (7160, 834)


100%|██████████| 7160/7160 [00:15<00:00, 459.18it/s]

iqs mean: 0.9626951593971564


In [79]:
kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=2022)
fold = 0
_x = X[X.index.isin(Y_train.index)].to_numpy()
for train_index, test_index in kf.split(_x):
  fold += 1
  print(f"Fold: {fold}")
  y = _x[test_index]
  for missing_perc in [0.01,
                       0.05,
                       0.1,
                       0.2]:
    print(f"Missing rate: {missing_perc}")
    file_name = f"./experimental_results/hla/maf/STI_wo_emb/preds_mixed_mr_{missing_perc}_rs_{fold}_.csv"
    result_genotypes = pd.read_csv(file_name, index_col=0)
    result_genotypes = result_genotypes.replace({
        '0|0': 0,
        '0|1': 1,
        '1|0': 1,
        '1|1': 2
    })
    yhat = result_genotypes.to_numpy()
    assert y.shape == yhat.shape
    iqs_a, iqs_mean_a = iqs_score(yhat, y)

    # print('iqs_mean:', iqs_mean_a)
    # print('iqs:', iqs_a)

    dataframe = pd.DataFrame(np.stack([np.full((len(iqs_a)), fold),
                                       np.full((len(iqs_a)), missing_perc),
                                       np.array(iqs_a),
                                       np.full((len(iqs_a)), "STI*WE"),
                                       np.array(bin_labels_str)], axis=1), columns=columns)
    dfs.append(dataframe)

Fold: 1
Missing rate: 0.01
data_imp: (835, 7161)
data_imp_012: (7161, 835)
snp_cnt: 7161
sample_cnt: 835
data_obs_012: (7161, 835)


100%|██████████| 7161/7161 [00:15<00:00, 450.81it/s]


iqs mean: 0.9982221594188123
Missing rate: 0.05
data_imp: (835, 7161)
data_imp_012: (7161, 835)
snp_cnt: 7161
sample_cnt: 835
data_obs_012: (7161, 835)


100%|██████████| 7161/7161 [00:15<00:00, 449.53it/s]


iqs mean: 0.9970456277057649
Missing rate: 0.1
data_imp: (835, 7161)
data_imp_012: (7161, 835)
snp_cnt: 7161
sample_cnt: 835
data_obs_012: (7161, 835)


...

iqs mean: 0.99697693277458
Missing rate: 0.1
data_imp: (834, 7161)
data_imp_012: (7161, 834)
snp_cnt: 7161
sample_cnt: 834
data_obs_012: (7161, 834)


100%|██████████| 7161/7161 [00:15<00:00, 453.22it/s]


iqs mean: 0.9954974629979897
Missing rate: 0.2
data_imp: (834, 7161)
data_imp_012: (7161, 834)
snp_cnt: 7161
sample_cnt: 834
data_obs_012: (7161, 834)


100%|██████████| 7161/7161 [00:15<00:00, 453.14it/s]

iqs mean: 0.9923200583538639


In [80]:
kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=2022)
fold = 0
_x = X[X.index.isin(Y_train.index)].to_numpy()
for train_index, test_index in kf.split(_x):
  fold += 1
  print(f"Fold: {fold}")
  y = _x[test_index]
  for missing_perc in [
                        0.01,
                       0.05,
                       0.1,
                       0.2]:
    print(f"Missing rate: {missing_perc}")
    file_name = f"./experimental_results/hla/maf/STI/preds_mixed_mr_{missing_perc}_rs_{fold}_.csv"
    result_genotypes = pd.read_csv(file_name, index_col=0)
    result_genotypes = result_genotypes.replace({
        '0|0': 0,
        '0|1': 1,
        '1|0': 1,
        '1|1': 2
    })
    yhat = result_genotypes.to_numpy()
    assert y.shape == yhat.shape
    iqs_a, iqs_mean_a = iqs_score(yhat, y)

    # print('iqs_mean:', iqs_mean_a)
    # print('iqs:', iqs_a)

    dataframe = pd.DataFrame(np.stack([np.full((len(iqs_a)), fold),
                                       np.full((len(iqs_a)), missing_perc),
                                       np.array(iqs_a),
                                       np.full((len(iqs_a)), "STI"),
                                       np.array(bin_labels_str)], axis=1), columns=columns)
    dfs.append(dataframe)

df = pd.DataFrame(columns = columns)
print("Combining the results...")
df = pd.concat(dfs, ignore_index=True)
del dfs

df

Fold: 1
Missing rate: 0.01
data_imp: (835, 7161)
data_imp_012: (7161, 835)
snp_cnt: 7161
sample_cnt: 835
data_obs_012: (7161, 835)


100%|██████████| 7161/7161 [00:15<00:00, 452.11it/s]


iqs mean: 0.99781957100763
Missing rate: 0.05
data_imp: (835, 7161)
data_imp_012: (7161, 835)
snp_cnt: 7161
sample_cnt: 835
data_obs_012: (7161, 835)


100%|██████████| 7161/7161 [00:15<00:00, 451.09it/s]


iqs mean: 0.9971245894365086
Missing rate: 0.1
data_imp: (835, 7161)
data_imp_012: (7161, 835)
snp_cnt: 7161
sample_cnt: 835
data_obs_012: (7161, 835)


...

100%|██████████| 7161/7161 [00:15<00:00, 450.35it/s]


iqs mean: 0.9962536907369116
Missing rate: 0.2
data_imp: (834, 7161)
data_imp_012: (7161, 834)
snp_cnt: 7161
sample_cnt: 834
data_obs_012: (7161, 834)


100%|██████████| 7161/7161 [00:15<00:00, 453.18it/s]


iqs mean: 0.9943829204671665
Combining the results...


,Fold,Missing Rate,IQS,Method,MAF
0,1,0.01,1.0,Minimac4,$0 \le MAF < 0.1 $
1,1,0.01,1.0,Minimac4,$0.3 \le MAF < 0.4 $
2,1,0.01,1.0,Minimac4,$0 \le MAF < 0.1 $
3,1,0.01,0.9950215830769596,Minimac4,$0 \le MAF < 0.1 $
4,1,0.01,1.0,Minimac4,$0 \le MAF < 0.1 $
...,...,...,...,...,...
515551,3,0.2,0.9975396705990637,STI,$0.1 \le MAF < 0.2 $
515552,3,0.2,0.9976034827085818,STI,$0.1 \le MAF < 0.2 $
515553,3,0.2,0.9981237134275976,STI,$0.3 \le MAF < 0.4 $
515554,3,0.2,1.0,STI,$0.1 \le MAF < 0.2 $


In [81]:
y.shape, yhat.shape, result_genotypes.shape

((834, 7161), (834, 7161), (834, 7161))

In [82]:
    df["IQS"] = df["IQS"].astype(float)
    df = df.round({'IQS': 5})
df

,Fold,Missing Rate,IQS,Method,MAF
0,1,0.01,1.00000,Minimac4,$0 \le MAF < 0.1 $
1,1,0.01,1.00000,Minimac4,$0.3 \le MAF < 0.4 $
2,1,0.01,1.00000,Minimac4,$0 \le MAF < 0.1 $
3,1,0.01,0.99502,Minimac4,$0 \le MAF < 0.1 $
4,1,0.01,1.00000,Minimac4,$0 \le MAF < 0.1 $
...,...,...,...,...,...
515551,3,0.2,0.99754,STI,$0.1 \le MAF < 0.2 $
515552,3,0.2,0.99760,STI,$0.1 \le MAF < 0.2 $
515553,3,0.2,0.99812,STI,$0.3 \le MAF < 0.4 $
515554,3,0.2,1.00000,STI,$0.1 \le MAF < 0.2 $


In [4]:
# df.to_csv("results_in_table/hla/maf/IQS_all.csv")
df = pd.read_csv("results_in_table/hla/maf/IQS_all.csv", index_col=0)
df

,Fold,Missing Rate,IQS,Method,MAF
0,1,0.01,1.00000,Minimac4,$0 \le MAF < 0.1 $
1,1,0.01,1.00000,Minimac4,$0.3 \le MAF < 0.4 $
2,1,0.01,1.00000,Minimac4,$0 \le MAF < 0.1 $
3,1,0.01,0.99502,Minimac4,$0 \le MAF < 0.1 $
4,1,0.01,1.00000,Minimac4,$0 \le MAF < 0.1 $
...,...,...,...,...,...
515551,3,0.20,0.99754,STI,$0.1 \le MAF < 0.2 $
515552,3,0.20,0.99760,STI,$0.1 \le MAF < 0.2 $
515553,3,0.20,0.99812,STI,$0.3 \le MAF < 0.4 $
515554,3,0.20,1.00000,STI,$0.1 \le MAF < 0.2 $


In [5]:
df1 = df.groupby(["Method", "Fold", "Missing Rate"]).agg( {"IQS":"mean"})
df1

IQS
Method Fold Missing Rate          
AE     1    0.01          0.995717
            0.05          0.988275
            0.10          0.980755
            0.20          0.961768
       2    0.01          0.995739
...                            ...
STI*WE 2    0.20          0.992014
       3    0.01          0.998093
            0.05          0.996977
            0.10          0.995497
            0.20          0.992320

[72 rows x 1 columns]

In [6]:
df1 = df.groupby(["Method", "Fold", "Missing Rate"]).agg( {"IQS":"mean"}).reset_index()# df1.columns = df1.columns.droplevel(0)
df1 = df1.groupby(["Method", "Missing Rate"]).agg( {"IQS":["mean", "std"]})#.reset_index()
df1


IQS          
                           mean       std
Method   Missing Rate                    
AE       0.01          0.995743  0.000028
         0.05          0.988089  0.000166
         0.10          0.980705  0.000238
         0.20          0.961948  0.000676
HLA*DEEP 0.01          0.987876  0.000494
         0.05          0.987339  0.000445
         0.10          0.986607  0.000388
         0.20          0.984855  0.000269
Minimac4 0.01          0.996462  0.000073
         0.05          0.996016  0.000085
         0.10          0.995412  0.000111
         0.20          0.994079  0.000155
SCDA+    0.01          0.996501  0.000149
         0.05          0.995374  0.000166
         0.10          0.993955  0.000199
         0.20          0.990838  0.000158
STI      0.01          0.997835  0.000034
         0.05          0.997151  0.000023
         0.10          0.996228  0.000027
         0.20          0.994300  0.000072
STI*WE   0.01          0.998156  0.000065
         0.05          0.996971  0.000078
         0.10          0.995465  0.000168
         0.20          0.992309  0.000289

In [9]:
# df1 = df.groupby(["Fold", "Missing Rate", "Method", "MAF"]).agg( {"IQS":"mean"}).reset_index()# df1.columns = df1.columns.droplevel(0)
df1 = df.groupby(["Missing Rate", "MAF", "Method"]).agg( {"IQS":"mean"}).reset_index()
df1["IQS"] = df1["IQS"].astype(float)
df1 = df1.round({'IQS': 4})
df1


,Missing Rate,MAF,Method,IQS
0,0.01,$0 \le MAF < 0.1 $,AE,0.9941
1,0.01,$0 \le MAF < 0.1 $,HLA*DEEP,0.9843
2,0.01,$0 \le MAF < 0.1 $,Minimac4,0.9964
3,0.01,$0 \le MAF < 0.1 $,SCDA+,0.9957
4,0.01,$0 \le MAF < 0.1 $,STI,0.9971
...,...,...,...,...
115,0.20,$0.4 \le MAF \le 0.5 $,HLA*DEEP,0.9846
116,0.20,$0.4 \le MAF \le 0.5 $,Minimac4,0.9940
117,0.20,$0.4 \le MAF \le 0.5 $,SCDA+,0.9907
118,0.20,$0.4 \le MAF \le 0.5 $,STI,0.9944
